<a href="https://colab.research.google.com/github/GeorgeNishanth/NG/blob/main/classificationproblem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Load the pre-trained model
pretrained_model = TfidfVectorizer()
pretrained_model.fit(['This is a positive review', 'This is a negative review'])
# Extract features from the data
X_train = pretrained_model.transform(['I love this product!', 'This product is terrible.'])

# Train a new model on the target task
y_train = np.array([1, 0])
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Make predictions on new data
X_new = pretrained_model.transform(['This product is the best!'])
y_pred = clf.predict(X_new)

# Print the prediction
print(y_pred)

[0]


In [ ]:
import torch
from sklearn.model_selection import train_test_split

# Example synthetic data (input texts and labels)
texts = [
    "I love this product!",  # Positive sentiment
    "This product is terrible.",  # Negative sentiment
    "I had a great experience.",  # Positive sentiment
    "I would not recommend this.",  # Negative sentiment
    "Absolutely fantastic!",  # Positive sentiment
    "Horrible experience.",  # Negative sentiment
]
labels = [1, 0, 1, 0, 1, 0]  # 1 for positive, 0 for negative

# Tokenize the texts
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Prepare the dataset
input_encodings = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
train_data = [{
    'input_ids': input_encodings['input_ids'][i],
    'attention_mask': input_encodings['attention_mask'][i],
    'labels': torch.tensor(labels[i])
} for i in range(len(labels))]

# Save the synthetic dataset
torch.save(train_data, "train_data.pt")


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.optim as optim

# Define the custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Load the synthetic dataset
train_data = torch.load("train_data.pt", weights_only=True)  # Use weights_only=True

# Create DataLoader
train_dataset = CustomDataset(train_data)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# Load the pre-trained model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Define the optimizer
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

# Training Loop
model.train()
for epoch in range(5):
    for batch in train_dataloader:

        input_ids = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)
        labels = batch['labels'].to(model.device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}: Loss = {loss.item()}")

# Save the fine-tuned model
model.save_pretrained("fine-tuned-model")
tokenizer.save_pretrained("fine-tuned-model")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1: Loss = 0.6838124990463257
Epoch 2: Loss = 0.7175323963165283
Epoch 3: Loss = 0.5076355338096619
Epoch 4: Loss = 0.41667771339416504
Epoch 5: Loss = 0.2675507068634033


('fine-tuned-model/tokenizer_config.json',
 'fine-tuned-model/special_tokens_map.json',
 'fine-tuned-model/vocab.txt',
 'fine-tuned-model/added_tokens.json',
 'fine-tuned-model/tokenizer.json')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the fine-tuned model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("fine-tuned-model")
tokenizer = AutoTokenizer.from_pretrained("fine-tuned-model")

# Set the model to evaluation mode
model.eval()

# Classify a text sequence
text_sequence = "This is a positive review."
inputs = tokenizer(text_sequence, return_tensors="pt", padding=True, truncation=True)

with torch.no_grad():
    outputs = model(**inputs)

# Get the predicted label
predicted_label = outputs.logits.argmax(-1).item()

# Print the predicted label
print(f"Predicted label: {predicted_label}")


Predicted label: 0
